# 서브워드 토크나이저 (Subword Tokenizer)
- OOV문제 해결방법 (회귀단어, 신조어)
- 하나의 단어를 여러 서브워드로 분리해서 인코딩 및 임베딩

## 바이트 페어 인코딩 (Byte Pair Encoding, BPE)
- 글자 단위에서 점차적으로 단어집합을 만든다 (Bottom up)

- 학습과정
1. 카운트 딕셔너리(일단 모든 단어를 글자단위로 분리)와 bpe_codes(merge 기록) 준비
2. 카운트를 통해 가장 많은 쌍을 찾고 딕셔너리에 쌍을 합치고 합친것을 bpe_codes에 추가
3. 2번 과정을 임의의 숫자 만큼 반복

- 테스트 과정
1. oov단어를 먼저 글자단위로 분리
2. bpe_codes에 쌍이 없을때까지 반복해서 merge

### 실습

In [1]:
import re, collections
from IPython.display import display, Markdown, Latex

# 반복횟수 지정
num_merges = 10 

# <\w>는 단어 맨끝에 붙이는 특수문자
dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

In [2]:
# pair들의 빈도수 카운트
def get_stats(dictionary):
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i], symbols[i+1]] += freq
    print('현재 pair들의 빈도수: ', dict(pairs))
    return pairs

In [6]:
pairs = get_stats(dictionary)
pair = max(pairs, key=pairs.get)
re.escape(' '.join(pair))

현재 pair들의 빈도수:  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}


'e\\ s'

In [7]:
# dictionary에서 가장 빈도수 큰 쌍 합치기
def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [9]:
bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i+1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)
    
    bpe_codes[best] = i
    bpe_codes_reverse[best[0]+best[1]] = best
    
    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수:  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수:  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수:  {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수:  {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수:  {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [10]:
bpe_codes

{('e', 's'): 0,
 ('es', 't'): 1,
 ('est', '</w>'): 2,
 ('l', 'o'): 3,
 ('lo', 'w'): 4,
 ('n', 'e'): 5,
 ('ne', 'w'): 6,
 ('new', 'est</w>'): 7,
 ('low', '</w>'): 8,
 ('w', 'i'): 9}

### WordPiece Tokenizer
- 우도(Likehood)가 가장 높은 거 병합 <-> BPE(빈도수)

## 센텐스피스 (SentencePiece)
- 사전 토큰화 작업 없이 단어 분리 토큰화 수행

### 실습

In [1]:
# !pip install sentencepiece

     ---------------------------------------- 1.1/1.1 MB 9.0 MB/s eta 0:00:00


In [2]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv
from pathlib import Path

data_dir = Path('C:/Users/011/sinjy1203/data')
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename=data_dir / "IMDb_Reviews.csv")

(WindowsPath('C:/Users/011/sinjy1203/data/IMDb_Reviews.csv'),
 <http.client.HTTPMessage at 0x18f4194c488>)

In [3]:
train_df = pd.read_csv(data_dir / 'IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [5]:
with open(data_dir / 'imdb_review.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_df['review']))

In [9]:
spm.SentencePieceTrainer.Train(
    '--input={} --model_prefix={} --vocab_size=5000 --model_type=bpe \
    --max_sentence_length=9999'.format(str(data_dir/'IMDb_Reviews.csv'), str(data_dir / 'imdb')))

In [11]:
vocab_list = pd.read_csv(data_dir / 'imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.head()

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,▁t,0
4,▁a,-1


In [12]:
vocab_list.shape

(5000, 2)

In [14]:
sp = spm.SentencePieceProcessor()
vocab_file = 'imdb.model'
sp.load(str(data_dir / vocab_file))

True

In [15]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[43, 629, 4951, 4926, 138, 172, 383, 30, 58, 74, 419, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[43, 142, 1371, 1123, 4, 674, 288, 93, 1085, 33, 92]

